In [10]:
import slate
import urllib
import re

In [3]:
furl = urllib.urlopen('http://www.nature.com/ismej/journal/v10/n1/pdf/ismej2015100a.pdf')
with open( '/tmp/asdfasdfasdf.pdf', 'w' ) as ftempfile :
    ftempfile.write( furl.read() )
with open( '/tmp/asdfasdfasdf.pdf' ) as f :
    doc = slate.PDF(f)

### PDF is garbage

In this example, we are looking for a link to some source code :

[`http://prodege.jgi-psf.org//downloads/src`](http://prodege.jgi-psf.org//downloads/src)

However, in the PDF, the URL is line wrapped, so the `src` is lost.

In [20]:
urlre = re.compile( '(?P<url>https?://[^\s]+)' )

for page in doc :
    print urlre.findall( page )

[]
['http://prodege.jgi-psf.org//downloads/', 'http://prodege.jgi-psf.org,']
[]
['http://img.jgi.', 'http://www.nature.com/ismej)']


### PDF is garbage, continued

If we remove line breaks to fix URLs that have been wrapped, we discover
that the visible line breaks in the document do not correspond to actual
line breaks in the represented text. The result is random garbage. 

In [19]:
urlre = re.compile( '(?P<url>https?://[^\s]+)' )

for page in doc :
    print urlre.findall( page.replace('\n','') )

[]
['http://prodege.jgi-psf.org//downloads/availablerun', 'http://prodege.jgi-psf.org,which']
[]
['http://img.jgi.Cell', 'http://creativecommons.org/licenses/by/4.0/the', 'http://www.nature.com/ismej)The']


At this point, the author elects to flip a table.